In [1]:
import json
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import keras
import nltk
stopwords = nltk.corpus.stopwords


Using TensorFlow backend.


In [7]:
train = pd.read_csv('train.csv', header=0, encoding='utf-8')
test = pd.read_csv('test.csv', header=0, encoding='utf-8')

In [3]:
print(train.shape)
print(test.shape)

(404290, 6)
(2345796, 3)


In [4]:
print(train.question1[0])
print(train.is_duplicate[0])

What is the step by step guide to invest in share market in india?
0


In [8]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from itertools import chain



In [9]:
f = open('glove.6B/glove.6B.50d.txt','r')
glove = {}
for line in f:
    split_line = line.split()
    word = split_line[0]
    vector = [float(val) for val in split_line[1:]]
    glove[word] = vector
gv = pd.DataFrame.from_dict(glove)
gv.head()    

,!,!!,!!!,!!!!,!!!!!,!?,!?!,"""",#,##,...,門下省,魏博,鳳翔,법정동,행정동,ﬁeld,ﬁgures,ﬁnds,ﬁrst,￥
0,-0.58402,-1.058800,0.115910,-0.245140,0.04081,0.150150,-0.55501,0.25769,-1.59250,-2.00650,...,-1.71560,-1.684200,-1.28810,0.23964,-0.23290,-0.308410,-1.26710,0.32301,0.387150,-0.019789
1,0.39031,0.269520,0.044932,0.077696,-0.17072,-0.436130,0.22278,0.45629,0.75569,-0.82787,...,-0.51879,-1.094600,-1.68480,-0.33579,-0.28264,-0.029643,-1.35540,0.32305,0.098414,-0.040119
2,0.65282,0.946320,0.669260,0.315350,1.12730,0.652120,0.15826,-0.76974,1.49470,0.26010,...,0.19944,-0.076817,-0.26558,0.52571,0.45301,-0.436600,0.18851,-0.31360,-0.738570,0.868910
3,-0.34030,0.056907,-0.678440,-0.980940,-0.80783,-0.078575,-0.52568,-0.37679,0.11055,0.40259,...,-0.84189,0.743410,0.57310,-0.51157,-0.30687,-0.039812,-1.19030,-0.20686,-0.732890,-0.440600
4,0.19493,0.244400,0.472530,0.292240,0.40568,0.214620,-0.04763,0.59272,-0.05244,-0.42902,...,-0.99399,-0.480240,-1.16220,-1.25120,-0.89357,0.080309,-0.42501,-0.39680,0.439770,-0.648040


In [10]:
# def tokenize_sentence(sentence):
#     words = nltk.word_tokenize(str(sentence).lower())
#     words = [word.lower() for word in words if word.isalpha()]
#     return words

# def glove_sentence(sentence):
#     return [gv[word] for word in sentence if word in gv]

# def sent_col_to_features(panda_col):
#     # Split sentence into words and convert to glove value
#     tokenized_sentences = panda_col.apply(lambda sentence: tokenize_sentence(sentence))
#     gloved_sentences = tokenized_sentences.apply(lambda sentence: glove_sentence(sentence))
#     # Take sum and mean of each sentence
#     return gloved_sentences.apply(lambda sentence: pd.Series([np.sum(sentence), np.mean(sentence)])).fillna(0)
# #     return gloved_sentences.apply(lambda sentence: np.sum(sentence)).fillna(0)

def tokenize_sentence(sentence):
    return nltk.word_tokenize(str(sentence).lower())

def glove_sentence(sentence):
    return [gv[word] for word in sentence if word in gv]

def get_sentence_features(sentence):
    return pd.Series([np.sum(sentence), np.mean(sentence)]).fillna(0)

def sent_col_to_features(panda_col):
    # Take sum and mean of each sentence
    return panda_col.apply(lambda sentence: get_sentence_features(glove_sentence(tokenize_sentence(sentence)))) 

In [110]:
num_sentences = len(train)
question_one_features = sent_col_to_features(train.question1[:num_sentences])
question_two_features = sent_col_to_features(train.question2[:num_sentences])
# question_one_features = sent_col_to_features(train.question1[:num_sentences])
# question_two_features = sent_col_to_features(train.question2[:num_sentences])

/Users/devLiu/miniconda2/envs/aind/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [109]:
print(question_one_features.shape)
print(question_two_features.shape)

(10000, 2)
(10000, 2)
404290
2345796


In [112]:
features = pd.concat([question_one_features , question_two_features],axis=1)
# features = np.hstack((question_one_features , question_two_features))
print(features.shape)
# features = features.values.reshape(num_sentences, 4)
labels = train.is_duplicate[:num_sentences].values.reshape(num_sentences, 1)

(404290, 4)


In [113]:
print(features.shape)
print(labels.shape)

(404290, 4)
(404290, 1)


In [114]:
# Split the pickled data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.3)

In [15]:
from sklearn import linear_model
clf_linear = linear_model.SGDClassifier()
clf_linear.fit(X_train, y_train)

/Users/devLiu/miniconda2/envs/aind/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [115]:
from sklearn.svm import SVC
clf_svc = SVC()
clf_svc.fit(X_train, y_train)

/Users/devLiu/miniconda2/envs/aind/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [2]:
import pickle
filename = 'svc_model.p'
pickle.dump(clf_svc, open(filename, 'wb'))

NameError: name 'clf_svc' is not defined

In [5]:
loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_val, y_val)
# print(result)

In [12]:
sent_col_to_features(test[:10]).to_pickle('test_features.p')

In [13]:
test_pickle = pd.read_pickle('test_features.p')
print(test_pickle)

     test_id   question1   question2
0  92.257261  263.927671  272.219359
1   0.076881    0.046303    0.051851


In [11]:
test_array = test
test_test_size = len(test)
%timeit q1_test = sent_col_to_features(test_array.question1[:test_test_size])
q2_test = sent_col_to_features(test_array.question2[:test_test_size])
%timeit test_features = pd.concat([q1_test , q2_test],axis=1)

/Users/devLiu/miniconda2/envs/aind/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


KeyboardInterrupt: 

In [73]:
predictions = clf_svc.predict(test_features)

# print_set = zip(predictions[:test_test_size], train.is_duplicate[:test_test_size])
# for guess, actual in print_set:
#     print("Prediction: ", guess, " Actual: ", actual)

# clf.score(X_val, y_val)

NameError: name 'clf' is not defined

In [ ]:
submission_predictions = pd.DataFrame(predictions[:10], columns=['is_duplicate'])
df.to_csv("submission.csv")